# CIP as a Service Data Challenge Demo

In [1]:
import requests
import json
import ipywidgets as widgets
from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl
import geopandas

In [2]:
req_domains = requests.get("https://cip-api.dmap-stage.aws.epa.gov/rpc/cipsrv_domains", verify = False)
req_domains

C:\Users\EKeith\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cip-api.dmap-stage.aws.epa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [200]>

In [3]:
req_domains.json()

{'states': [{'name': 'Alaska', 'geoid': '02', 'stusps': 'AK'},
  {'name': 'Alabama', 'geoid': '01', 'stusps': 'AL'},
  {'name': 'Arkansas', 'geoid': '05', 'stusps': 'AR'},
  {'name': 'American Samoa', 'geoid': '60', 'stusps': 'AS'},
  {'name': 'Arizona', 'geoid': '04', 'stusps': 'AZ'},
  {'name': 'California', 'geoid': '06', 'stusps': 'CA'},
  {'name': 'Colorado', 'geoid': '08', 'stusps': 'CO'},
  {'name': 'Connecticut', 'geoid': '09', 'stusps': 'CT'},
  {'name': 'District of Columbia', 'geoid': '11', 'stusps': 'DC'},
  {'name': 'Delaware', 'geoid': '10', 'stusps': 'DE'},
  {'name': 'Florida', 'geoid': '12', 'stusps': 'FL'},
  {'name': 'Georgia', 'geoid': '13', 'stusps': 'GA'},
  {'name': 'Guam', 'geoid': '66', 'stusps': 'GU'},
  {'name': 'Hawaii', 'geoid': '15', 'stusps': 'HI'},
  {'name': 'Iowa', 'geoid': '19', 'stusps': 'IA'},
  {'name': 'Idaho', 'geoid': '16', 'stusps': 'ID'},
  {'name': 'Illinois', 'geoid': '17', 'stusps': 'IL'},
  {'name': 'Indiana', 'geoid': '18', 'stusps': 'IN'

In [4]:
clip_select = []
for st in req_domains.json()["states"]:
    clip_select.append((st["name"], "USPS:" + st["stusps"]))

for tr in req_domains.json()["tribes"]:
    clip_select.append((tr["aiannha_namelsad"], "AIANNHNS:" + tr["aiannha_geoid_stem"] + ":T"))

In [52]:
### Pretty much taken directly from the ipyleaflet documentation to allow users to input geometries
m = Map(
    basemap=basemap_to_tiles(basemaps.OpenTopoMap),
    center=(38.8, -77.0),
    zoom=12
    )

draw_control = DrawControl()
draw_control.polyline =  {
    "shapeOptions": {
        "color": "#6bc2e5",
        "weight": 8,
        "opacity": 1.0
    }
}
draw_control.polygon = {
    "shapeOptions": {
        "fillColor": "#6be5c3",
        "color": "#6be5c3",
        "fillOpacity": 0.6
    },
    "drawError": {
        "color": "#dd253b",
        "message": "Oups!"
    },
    "allowIntersection": False
}
draw_control.circle = {
    "shapeOptions": {
        "fillColor": "#efed69",
        "color": "#efed69",
        "fillOpacity": 1.0
    }
}
draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 1.0
    }
}

feature_collection = {
    'type': 'FeatureCollection',
    'features': []
}

def handle_draw(self, action, geo_json):
    """Save the GeoJSON when it's drawn on the map"""
    feature_collection['features'].append(geo_json)
    
draw_control.on_draw(handle_draw)

m.add_control(draw_control)

m

Map(center=[38.8, -77.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

In [53]:
nhd_version = widgets.ToggleButtons(
    options = [("Medium Res (v2.1)", "nhdplus_m"), ("High Res", "nhdplus_h")],
    description = "NHDPlus Version"
)
clip_to = widgets.Dropdown(
    options=clip_select,
    description='Where to clip to',
    style=dict(description_width='initial')
)
clip_when = widgets.ToggleButtons(
    options = [("Before", "BEFORE"), ("After", "AFTER")],
    description = "When to clip"
)
catchment_filter = widgets.Dropdown(
    options = [(st["name"], st["stusps"]) for st in req_domains.json()["states"]],
    description = "Catchment filter",
    style=dict(description_width='initial')
)
default_point_indexing_method = widgets.ToggleButtons(
    options = [("Simple", "point_simple")],
    description = "Default point indexing method",
    value = "point_simple"
)
default_line_indexing_method = widgets.ToggleButtons(
    options = [("Simple", "line_simple"), ("Level Path", "line_levelpath")],
    description = "Default line indexing method",
    value = "line_levelpath"
)
default_ring_indexing_method = widgets.ToggleButtons(
    options = [("Simple", "area_simple"), ("Centroid", "area_centroid"), ("Artificial Path", "area_artpath"), ("Treat rings as lines", "treat_as_lines")],
    description = "Default ring indexing method",
    value = "area_simple"
)
default_area_indexing_method = widgets.ToggleButtons(
    options = [("Simple", "area_simple"), ("Centroid", "area_centroid"), ("Artificial Path", "area_artpath")],
    description = "Default area indexing method",
    value = "area_simple"
)
default_line_threshold = widgets.IntSlider(
    value=10,
    min=0,
    max=100,
    description='Line threshold',
    style=dict(description_width='initial')
)
default_areacat_threshold = widgets.IntSlider(
    value=50,
    min=0,
    max=100,
    description='Area Catchment threshold',
    style=dict(description_width='initial')
)
default_areaevt_threshold = widgets.IntSlider(
    value=1,
    min=0,
    max=100,
    description='Area Catchment threshold',
    style=dict(description_width='initial')
)
return_indexed_features = widgets.ToggleButton(
    value=True,
    description='Return indexed features'
)
return_indexed_collection = widgets.ToggleButton(
    value=False,
    description='Return indexed collection'
)
return_catchment_geometry = widgets.ToggleButton(
    value=True,
    description='Return catchment geometry'
)
return_flowlines = widgets.ToggleButton(
    value=True,
    description='Return flowlines'
)
return_flowline_geometry = widgets.ToggleButton(
    value=True,
    description='Return flowline geometry'
)
return_huc12s = widgets.ToggleButton(
    value=False,
    description='Return HUC12s'
)
# "wbd_version":null

In [38]:
display(nhd_version)
display(clip_to)
display(clip_when)
display(catchment_filter)
display(default_point_indexing_method)
display(default_line_indexing_method)
display(default_ring_indexing_method)
display(default_area_indexing_method)
display(default_line_threshold)
display(default_areacat_threshold)
display(default_areaevt_threshold)
display(return_indexed_features)
display(return_indexed_collection)
display(return_catchment_geometry)
display(return_flowlines)
display(return_flowline_geometry)
display(return_huc12s)

ToggleButtons(description='NHDPlus Version', options=(('Medium Res (v2.1)', 'nhdplus_m'), ('High Res', 'nhdplu…

Dropdown(description='Where to clip to', options=(('Alaska', 'USPS:AK'), ('Alabama', 'USPS:AL'), ('Arkansas', …

ToggleButtons(description='When to clip', options=(('Before', 'BEFORE'), ('After', 'AFTER')), value='BEFORE')

Dropdown(description='Catchment filter', options=(('Alaska', 'AK'), ('Alabama', 'AL'), ('Arkansas', 'AR'), ('A…

ToggleButtons(description='Default point indexing method', options=(('Simple', 'point_simple'),), value='point…

...

ToggleButton(value=False, description='Return indexed collection')

ToggleButton(value=True, description='Return catchment geometry')

ToggleButton(value=True, description='Return flowlines')

ToggleButton(value=True, description='Return flowline geometry')

ToggleButton(value=False, description='Return HUC12s')

In [39]:
input_data = {}
input_data["geometry"] = feature_collection
input_data["nhdplus_version"] = nhd_version.value
input_data["geometry_clip"] = clip_to.value
input_data["geometry_clip_stage"] = clip_when.value
input_data["catchment_filter"] = catchment_filter.value
input_data["default_point_indexing_method"] = default_point_indexing_method.value
input_data["default_line_indexing_method"] = default_line_indexing_method.value
input_data["default_ring_indexing_method"] = default_ring_indexing_method.value
input_data["default_area_indexing_method"] = default_area_indexing_method.value
input_data["default_line_threshold"] = default_line_threshold.value
input_data["default_areacat_threshold"] = default_areacat_threshold.value
input_data["default_areaevt_threshold"] = default_areaevt_threshold.value
input_data["return_indexed_features"] = return_indexed_features.value
input_data["return_indexed_collection"] = return_indexed_collection.value
input_data["return_catchment_geometry"] = return_catchment_geometry.value
input_data["return_flowlines"] = return_flowlines.value
input_data["return_flowline_geometry"] = return_flowline_geometry.value
input_data["return_huc12s"] = return_huc12s.value

In [40]:
input_data

{'geometry': {'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'properties': {'style': {'stroke': True,
      'color': '#6bc2e5',
      'weight': 8,
      'opacity': 1,
      'fill': False,
      'clickable': True}},
    'geometry': {'type': 'LineString',
     'coordinates': [[-77.032866, 38.819102],
      [-77.031492, 38.78432],
      [-77.036299, 38.756481]]}}]},
 'nhdplus_version': 'nhdplus_m',
 'geometry_clip': 'USPS:DC',
 'geometry_clip_stage': 'BEFORE',
 'catchment_filter': 'DC',
 'default_point_indexing_method': 'point_simple',
 'default_line_indexing_method': 'line_levelpath',
 'default_ring_indexing_method': 'area_simple',
 'default_area_indexing_method': 'area_simple',
 'default_line_threshold': 10,
 'default_areacat_threshold': 50,
 'default_areaevt_threshold': 1,
 'return_indexed_features': True,
 'return_indexed_collection': False,
 'return_catchment_geometry': True,
 'return_flowlines': True,
 'return_flowline_geometry': True,
 'return_huc12s': False}

In [41]:
req = requests.post("https://cip-api.dmap-stage.aws.epa.gov/rpc/cipsrv_index", data = json.dumps(input_data), verify = False)
req

C:\Users\EKeith\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cip-api.dmap-stage.aws.epa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [200]>

In [42]:
req.json()

{'huc12s': None,
 'flowlines': {'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'geometry': {'type': 'LineString',
     'coordinates': [[-77.032298985, 38.832049127],
      [-77.032146641, 38.829476793],
      [-77.033763948, 38.822081721],
      [-77.033288942, 38.820733588],
      [-77.03310613, 38.817646785],
      [-77.033364326, 38.816667385],
      [-77.033115184, 38.814462318],
      [-77.032005907, 38.811741383],
      [-77.032136769, 38.810614783],
      [-77.03112429, 38.807526582],
      [-77.032513468, 38.801625378]]},
    'properties': {'gnis_id': '597915',
     'fmeasure': 0.0,
     'tmeasure': 100.0,
     'gnis_name': 'Potomac River',
     'nhdplusid': 22340577,
     'reachcode': '02070010000046'}},
   {'type': 'Feature',
    'geometry': {'type': 'LineString',
     'coordinates': [[-77.032513468, 38.801625378],
      [-77.032644331, 38.800498777],
      [-77.032361258, 38.799053109],
      [-77.033264513, 38.795625174],
      [-77.032825635, 38.792880

In [43]:
gdf = geopandas.GeoDataFrame.from_features(req.json()['catchments']['features'], crs="EPSG:4326")
gdf

,geometry,areasqkm,nhdplusid,xwalk_huc12,catchmentstatecode
0,"POLYGON ((-77.03802 38.83076, -77.03800 38.830...",5.222295,22340577,020700100301,DC
1,"POLYGON ((-77.03876 38.80275, -77.03896 38.801...",0.618188,22340603,020700100307,DC
2,"POLYGON ((-77.03739 38.77660, -77.03534 38.776...",7.028703,22340603,020700100307,MD
3,"POLYGON ((-77.03670 38.75492, -77.03636 38.754...",3.799692,22340617,020700100307,MD
4,"POLYGON ((-77.04204 38.80217, -77.04102 38.802...",1.957906,22340577,020700100301,VA
5,"MULTIPOLYGON (((-77.04015 38.79043, -77.04004 ...",0.427008,22340603,020700100307,VA
6,"POLYGON ((-77.04146 38.76248, -77.04140 38.764...",0.671508,22340617,020700100307,VA


In [44]:
gdf.explore()